In [1]:
# set up the notebook

%load_ext autoreload
%autoreload 2

import sys
import os
import logging
logging.basicConfig(level=logging.ERROR)

# Preparation

This part will help you get access to OpenAI API and run your first completion.
- For more information about model size and pricing, see [this page](https://openai.com/api/pricing/).
- For a detailed API walkthrough (with examples), see [this documentation page](https://beta.openai.com/docs/api-reference/completions).

In [2]:
# get openAI access and see its model list

import openai
import json

with open("./credential.json", "r") as f:
    credential = json.load(f)["openai"]
openai.api_key = credential

# Test to see if you've got the access successfuly using the cheapest model.
completion = openai.Completion.create(
  model="ada",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)
completion

<OpenAIObject text_completion id=cmpl-6VDkMsaHlgFv5gkxufRa1nQt3O8uQ at 0x7fc750812590> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " of your ability to read the signs"
    }
  ],
  "created": 1672899030,
  "id": "cmpl-6VDkMsaHlgFv5gkxufRa1nQt3O8uQ",
  "model": "ada",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}

In [3]:
# If you got the completion successfully, you should be able to access the generated text through this line:
completion["choices"][0]["text"]

' of your ability to read the signs'

# Writeup 0: Task and Strategy Description

**EDIT THIS PART TO PROVIDE AN OVERVIEW OF YOUR ATTEMPTS**

- **Task Description**: Create three metaphors for a given concept, so that we can explain the different aspects of crowdsourcing in a poetic way. A metaphor may look like:

    > In crowdsourcing, people are like bees; they work together to make honey.

    With the concept being “crowdsourcing”, the simile being “bees”, and the similar aspect being “work together.”
    
    This task is from paper [AI Chains: Transparent and Controllable Human-AI Interaction
by Chaining Large Language Model Prompts](https://arxiv.org/pdf/2110.01691.pdf), Case 0 in B.3.
- **Example Input/output**: Write >=3 input-output pairs of your task. You should test your strategy on all the three examples.
    ```
    Input: Crowdsourcing
    Output: 
    1. crowdsourcing is like a team sport in that it brings people to achieve one goal.
    2. Crowdsourcing is like a vast, open-source library, with everyone contributing to the shelves of knowledge.
    3. Crowdsourcing was like a quilt, with each individual contributing a patch of fabric to create a larger piece of art.

    Input: Gratitude
    Output:
    1. gratitude is like a stream in that it’s a force that can carry you along.
    2. Gratitude is like a generous river that overflows with blessings.
    3. Gratitude is a feather in the cap of humility.

    Input: loss
    Output:
    1. loss is like a wing in that it’s something you never wanted to lose, and it can take you away.
    2. The loss was like a dark cloud of disappointment hovering over me.
    3. Loss is a heavy burden, weighing down the heart like a funeral shroud.
    ```
- **Workflow prompting strategy**: The pipeline helps further specify an abstract query into more specific aspects. Given an input concept, we will
    - First find three unique traits of the concept
    - Ask the model to write a different metaphor for each trait.
    
    For example:
    ```
    INPUT: gratitude
    Output: 
    TRAIT => OUTPUT: 
    Appreciation => Gratitude is a warm embrace, showing appreciation for all that is given.
    Generosity => Gratitude is like a generous river that overflows with blessings.
    ```
- **Crowdsourcing paper**: [Towards Large-Scale Collaborative Planning: Answering High-Level Search Queries Using Human Computation](https://ojs.aaai.org/index.php/AAAI/article/view/8092) -- where the pipelining strategy comes from.

## Coding: Use this part to do your prompting.

Tips for prompting:
- [OpenAI playground](https://beta.openai.com/playground) offers more interactive prompting experiments.
- Read the [API documentation](https://beta.openai.com/docs/api-reference/completions?lang=python) to use the right parameters (e.g., `temperature`, `max_length`).
- Read the [tutorial doc](https://beta.openai.com/docs/introduction) to get a general sense of GPT-3.
- Read [OpenAI examples](https://beta.openai.com/examples) to get a sense of what GPT-3 can do.

Tips for saving credits:

- You can use a cheaper model for tuning your prompts (see [pricing](https://openai.com/api/pricing/)) before generating the final responses using the most capable model `text-davinci-003`
- [AI21](https://www.ai21.com/studio) provides a model similar to GPT-3 / API service similar to OpenAI, and it involves more free credits for you to try. If you choose to use this, remember to also get its API key and set up the credential.

## Baseline prompting

Use this section to do a one-step prompting for your selected task.

In [32]:
# Write your code here.

def baseline_prompt(concept, model="text-davinci-003", max_tokens=100, temperature=0.7, top_p=1):
    prompt = f"""The following is a list of three metaphors on '{concept}'.
Concept: {concept}
Metaphors: 1."""
    completion = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p
    )
    output = completion["choices"][0]["text"].split("4.")[0]
    print(f"INPUT: {concept}")
    print(f"OUTPUT: {output}")
    print()
    
baseline_prompt("crowdsourcing")

baseline_prompt("gratitude")

baseline_prompt("loss")

INPUT: crowdsourcing
OUTPUT:  Crowdsourcing is like a tapestry, with many individual threads weaving together to create a beautiful whole. 
2. Crowdsourcing is like a relay race, with each person taking the baton and running just a short distance before handing it off to another. 
3. Crowdsourcing is like a jigsaw puzzle, with many people contributing pieces to the final picture.

INPUT: gratitude
OUTPUT:  Gratitude is a garden that blossoms with love.
2. Gratitude is a river that carries abundance.
3. Gratitude is a beacon of light that guides the way.

INPUT: loss
OUTPUT:  Time is a thief that steals away everything precious. 
2. Life is a game of chess, and each move is a loss. 
3. Life is a rollercoaster ride, with moments of joy and sorrow.



## Crowdsourcing-Pipeline-inspired prompting

Use this section to your selected pipeline prompting for your task.

In [31]:
import re
re.split(r'\d+\.', "1. hi, 2.")

['', ' hi, ', '']

In [46]:
# Write your code here.
from functools import partial
import re

def pipeline_prompt(concept, model="text-davinci-003", max_tokens=100, temperature=0.7, top_p=1):
    prompt_concept_to_trait = f"""The following three phrases describes three different and unique traits of the given concept.
Concept: {concept}
Unique traits: 1."""
    completion = openai.Completion.create(
        model=model,
        prompt=prompt_concept_to_trait,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p
    )
    traits = completion["choices"][0]["text"].split("4.")[0]
    traits = [t.strip() for t in re.split(r'\d+\.', traits) if t.strip()]
    
    metaphors = []
    for trait in traits:
        prompt_trait_to_metaphor = f"""Given the concept and its unique trait, write a metaphor that conveys the corresponding trait.
Concept: {concept}
trait: {trait}
metaphor:"""
        completion = openai.Completion.create(
            model=model,
            prompt=prompt_trait_to_metaphor,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p
        )
        metaphors.append(completion["choices"][0]["text"].strip())
    metaphor = '\n'.join(metaphors)
    print(f"INPUT: {concept}")
    print(f"TRAITS: \n{traits}")
    print(f"OUTPUT: \n{metaphor}")
    print()

pipeline_prompt("crowdsourcing")

pipeline_prompt("gratitude")

pipeline_prompt("loss")



INPUT: crowdsourcing
TRAITS: 
['Collaborative problem-solving;', 'Open-source resource sharing;', 'Online collaboration.']
OUTPUT: 
Crowdsourcing is like a collective brain, offering an array of diverse perspectives to solve a single problem.
Crowdsourcing is like a vast, open-source library, with everyone contributing to the shelves of knowledge.
Crowdsourcing was like a quilt, with each individual contributing a patch of fabric to create a larger piece of art.

INPUT: gratitude
TRAITS: 
['Appreciation', 'Generosity', 'Humility']
OUTPUT: 
Gratitude is a warm embrace, showing appreciation for all that is given.
Gratitude is like a generous river that overflows with blessings.
Gratitude is a feather in the cap of humility.

INPUT: loss
TRAITS: 
['Bereavement', 'Disappointment', 'Regret']
OUTPUT: 
Loss is a heavy burden, weighing down the heart like a funeral shroud.
The loss was like a dark cloud of disappointment hovering over me.
Loss is like a heavy weight on my chest, crushing me wi

# Writeup 1: Report & Reflection

Fill in the following three sections by reflecting on your results.

## Report the result

(For all the inputs you tried, summarized the input, baseline output, pipeline output, which output you like and why.)

- **Input**:
- **Baseline output**:
- **Pipeline output**:
- **Which output you like and why**:



- **Input**:
- **Baseline output**:
- **Pipeline output**:
- **Which output you like and why**:

...

## Reflect on prompting effectiveness

Write some paragraphs to describe: Did you find the pipeline prompting workflow effective? Why or Why not?

## Envision possible improvements

Write some paragraphs to describe: What are some possible ways to further improve the pipeline or prompt design?